# Imports

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html5lib
import lxml
import re
from datetime import datetime
import time


# Setup

In [4]:
# OHSAA Southwest District conferences
domains = ['https://eccsports.com/',
           'https://gmcsports.com/',
           'http://www.swocsports.com/',
           'http://www.swblsports.com/',
           'http://sbaac.com/',
           'http://ggcl.gclsports.com/',
           ]

# Gender URL examples (standings, schedule)
girls = ['35', '215']
boys = ['30', '137']

# Requests headers
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'}


# Download List of Matches

In [6]:
match_ids = []
totals_rows_list = []
for domain in domains[:]:
    for gender in [girls, boys]:
        for year in range(2007, 2023):
            page = requests.get(
                f'{domain}schedule.aspx?satc={gender[1]}&cmp=1&year={year}', headers=headers)
            soup = BeautifulSoup(page.content, 'html.parser')
            title = soup.find('title').string
            match_links = soup.select('a[id*=ContentPlaceHolder1]')
            for link in match_links:
                href = link['href']
                matches = re.findall(r'id=(.*)', href)

                # Add to list of Match IDs
                match_ids.append(matches[0])

                # Add to list of Match IDs with totals from table
                id = int(matches[0])
                total = link.parent.find_previous_sibling().findChild().contents
                date = link.find_previous('th').find_previous('th').contents
                date_parsed = str(datetime.strptime(
                    date[0], '%A, %B %d, %Y').isoformat()[:10])

                row = [id, total, date_parsed,
                       f'{year}-{year+1}', gender[1], domain, title]
                totals_rows_list.append(row)
            time.sleep(10)


In [7]:
df_matches_totals = pd.DataFrame(totals_rows_list, columns=[
                                 'MatchID', 'Total', 'Date', 'Year', 'Gender', 'Domain', 'Title'])
df_matches_totals['Total'] = df_matches_totals['Total'].str[0]
df_matches_totals.to_csv('matches_sw_district.csv')


## Filter out matches without scores or outside usual

In [ ]:
has_score_filter = df_matches_totals['Total'].str.contains('(\d\d\d.*){2}')
df_has_score = df_matches_totals[has_score_filter]
not_actual_match = df_has_score['Total'].str.contains(
    '(\d\d\d.+){3,}|( Tournament )|( at )|(lassic)|(,.+){2}')
df_has_score = df_has_score[~not_actual_match]


In [9]:
df_has_score.to_csv('matches_sw_district_have_scores.csv')